# Natural Language Processing (NLP)
## Text Categorization

+ **Natural Language Processing (NLP)** is a discipline within AI that combines computing with human communication.
+ Applications are Speech Recognition, Speech Tagging, Machine Translation, Sentiment Analysis, Chatbots, Text Summarization and a multitude of other functions.


### References:
+ https://realpython.com/python-nltk-sentiment-analysis/
+ https://www.nltk.org/howto/classify.html
+ https://www.nltk.org/book/ch06.html
+ https://www.tensorflow.org/text/tutorials/classify_text_with_bert
+ [NLTK Summary](https://towardsdatascience.com/text-summarization-using-tf-idf-e64a0644ace3)
+ [How to summarize text with OpenAI and LangChain](https://medium.com/@johnidouglasmarangon/how-to-summarize-text-with-openai-and-langchain-e038fc922af)
+ [Text Summary with OpenAI GPT-3 API](https://medium.com/muthoni-wanyoike/implementing-text-summarization-using-openais-gpt-3-api-dcd6be4f6933)
    + [Improved Implementation with OpenAI](https://medium.com/@tanguyvans/how-to-summarize-long-texts-using-openai-improving-coherence-and-structure-d896c5510c45)
+ [Text Summary with Multiple Tools](https://medium.com/analytics-vidhya/text-summarization-using-bert-gpt2-xlnet-5ee80608e961)
+ [Text Summary with Llama2](https://medium.com/@tushitdavergtu/llama2-and-text-summarization-e3eafb51fe28)

In [ ]:
BUCKET_NAME ="cio-training-vertex-colab"
PROJECT_ID  ="usfs-ai-bootcamp"
LOCATION    = "us-central1"

In [ ]:
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#- Google Colab Check
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import datetime

RunningInCOLAB = False
RunningInCOLAB = 'google.colab' in str(get_ipython())
current_time   = datetime.datetime.now()

if RunningInCOLAB:
    print(f"You are running this notebook in Google Colab at {current_time} in the {PROJECT_ID} lab.")
else:
    print(f"You are likely running this notebook with Jupyter iPython runtime at {current_time} in the {PROJECT_ID} lab.")

## Library Management

In [ ]:

import sys
import subprocess
import importlib.util

In [ ]:
libraries=["nltk", "bs4", "wordcloud", "pathlib", "numpy", "Pillow", "pandas", "seaborn"]
import importlib.util

for library in libraries:
    if library == "Pillow":
      spec = importlib.util.find_spec("PIL")
    else:
      spec = importlib.util.find_spec(library)
    if spec is None:
      print("Installing library " + library)
      subprocess.run(["pip", "install" , library, "--quiet"])
    else:
      print("Library " + library + " already installed.")

In [ ]:

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#- Import Natural Language Processing (NLP) specific libs
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer  # A word stemmer based on the Porter stemming algorithm.  Porter, M. "An algorithm for suffix stripping." Program 14.3 (1980): 130-137.
from nltk import pos_tag
from nltk.tree import tree
from nltk import FreqDist

#from nltk.book import * #<- Large Download, only pull if you want raw material to work with

In [ ]:
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# More NLP specific libraries
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import nltk
from nltk.stem import *

from bs4 import BeautifulSoup                 #used to parse the text
from wordcloud import WordCloud, STOPWORDS    #custom library specifically designed to make word clouds
from nltk.stem import WordNetLemmatizer

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# a set of libraries that perhaps should always be in Python source
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import os
import socket
import sys
import getopt
import inspect
import warnings
import json
import pickle
from pathlib import Path
import itertools
import datetime
import re
import shutil
import string
import io

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Additional libraries for this work
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import math
from base64 import b64decode
from IPython.display import Image
import requests

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Data Science Libraries
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import numpy as np
import pandas as pd
import seaborn as sns

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Graphics
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import matplotlib.pyplot as plt
from PIL import Image
import PIL.ImageOps

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# progress bar
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
from tqdm import tqdm

In [ ]:
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#- NLTK required resources
#- Required to load necessary files to support NLTK
#- Downloads repository of knowledge to augment (this is the data portion) the library
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
nltk.download("stopwords")
nltk.download("words")
nltk.download('punkt')
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download('punkt_tab')
#nltk.download("all")  #<- Only do this if you want the full spectrum of all possible packages, it's a LOT!

# Noun Part of Speech Tags used by NLTK
# More can be found here
# http://www.winwaed.com/blog/2011/11/08/part-of-speech-tags/
#NOUNS = ['NN', 'NNS', 'NNP', 'NNPS']
#VERBS = ['VB', 'VBG', 'VBD', 'VBN', 'VBP', 'VBZ']

## Functions

In [ ]:
## Outputs library version history of effort.
#
def lib_diagnostics() -> None:

    import pkg_resources

    package_name_length=40
    package_version_length=20

    # Get installed packages
    the_packages=["nltk", "numpy", "os", "pandas", "seaborn"]
    installed = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
    for package_idx, package_name in enumerate(installed):
         if package_name in the_packages:
             installed_version = installed[package_name]
             print(f"{package_name:<40}#: {str(pkg_resources.parse_version(installed_version)):<20}")

    try:
        print(f"{'TensorFlow version':<40}#: {str(tf.__version__):<20}")
        print(f"{'     gpu.count:':<40}#: {str(len(tf.config.experimental.list_physical_devices('GPU')))}")
        print(f"{'     cpu.count:':<40}#: {str(len(tf.config.experimental.list_physical_devices('CPU')))}")
    except Exception as e:
        pass

    try:
        print(f"{'Torch version':<40}#: {str(torch.__version__):<20}")
        print(f"{'     GPUs available?':<40}#: {torch.cuda.is_available()}")
        print(f"{'     count':<40}#: {torch.cuda.device_count()}")
        print(f"{'     current':<40}#: {torch.cuda.current_device()}")
    except Exception as e:
        pass


    try:
      print(f"{'OpenAI Azure Version':<40}#: {str(the_openai_version):<20}")
    except Exception as e:
      pass
    return

## Function Call

In [ ]:
lib_diagnostics()

# Input Sources

In [ ]:
###########################################
#- API Parameters for things like WordCloud
#- Variables help hold information for later use
#- The "constants" represent variables that we don't anticipate changing over the course of the program.
###########################################
IMG_BACKGROUND=None                             #None without quotes or "black", "white", etc...
IMG_FONT_SIZE_MIN=10
IMG_WIDTH=1024
IMG_HEIGHT=768
FIGURE_WIDTH=11
FIGURE_HEIGHT=8.5
WORD_FREQ=10

In [ ]:
#!rm -rf ./folderOnColab && echo "Ok, removed." || { echo "No folder to remove."; exit 1; }
#!mkdir -p ./folderOnColab && echo "Folder created." || { echo "Failed to create folder, it might already exist.";  }
#!gsutil -m cp -r gs://usfs-gcp-rand-test-data-usc1/public_source/jbooks/ANewHope.txt ./folderOnColab

target_folder="./folderOnColab"
target_files=["ANewHope.txt", "slf*.txt", "alb*.txt"]
print(f"Creating a folder ({target_folder}) to store project data.")
subprocess.run(["mkdir", "-p" , target_folder])
if os.path.isdir(target_folder):
  for idx, filename in enumerate(target_files):
    print(f"Copying {filename} to target folder: {target_folder}")
    subprocess.run(["gsutil", "-m" , "cp", "-r", f"gs://{BUCKET_NAME}/training-data/jbooks/{filename}",  target_folder], check=True)
else:
    print("ERROR: Local folder not found/created.  Check the output to ensure your folder is created.")
    print(f"...target folder: {target_folder}")
    print("...if you can't find the problem contact the instructor.")


In [ ]:
data=""

#select the filename you want to process your body of text from: ANewHope.txt, slf_final_wordcloud_content.txt, alb_final_wordcloud_content.txt
target_filename=target_folder+os.sep+"slf_final_wordcloud_content.txt"          #<- Change here


#check for the file's existence
if os.path.isfile(target_filename):
  #open the file, read the contents and close the file
  f = open(target_filename, "r", encoding="cp1252")
  data=f.read()
  f.close()
else:
    print("ERROR: File not found.  Check the previous code block to ensure you file copied.")
    print(f"...target file: {target_filename}")
    print("...if you can't find the problem contact the instructor.")

if len(data)<1:
    print("ERROR: There is no content in your data variable.")
    print("...Verify you copied the input file correctly.")
    print("...if you can't find the problem contact the instructor.")
else:
    print(f"It appears your data file was read, your data file has {len(data):,} elements of data.")


# *Natural Language Processing Methods *

+ **Tokenization** is a fundamental task that involves breaking down a piece of text into smaller units called tokens. These tokens can be individual characters, entire words, or even subwold pieces, depending on the tokenization method used.
+ **Stop words** are inconsequential words that appear frequently in a language but don’t contribute much to the overall understanding of a sentence or document. They often include articles, prepositions, conjunctions, and common verbs.
+ **Stemming** is a text preprocessing technique that involves reducing words to their base or root form, known as stems.
+ **Lemmatization** aims to identify and convert words to their base forms by considering their grammatical properties and meanings.


***Why might a Data Scientist use NLP techniques on a body of text when performing AI work?***

In [ ]:
###########################################
#- Demonstrate use of tokens and stopwords
###########################################

#Perform a tokenization at the sentence level of the data.
response=sent_tokenize(data)
print(f"There are {len(response)} sentences.")

#extra code to debug and see how the output looks.
#for the_index, the_sentence in enumerate(response):
#    print(f"{the_index}. {the_sentence.strip()}")
#
#print("\n")

#Perform a tokenization at the word level of the data.
response=word_tokenize(data)
print(f"There are {len(response)} words.")

#apply stop words to remove inconsequential words that appear frequently but don't influence the overall understanding of the setences.
#gather the stop words for the NLTK library into a variable
stop_words = set(stopwords.words("english"))
#create a list data structure that will hold the resulting words
filtered_list = []
#break the overall data into "word" tokens after making everything lowercase (why would we do that?  Ask the instructor?)
word_token_response=word_tokenize(data.lower())

#Python "lamba" expression, very efficient for loop, used to continue normalizing the data by only allowing alpha characters that are equal to or greater than 2.
wordlist = [x for x in word_token_response if (len(x)>=2 and x.isalpha())]

#loop through each word in the wordlist and verify that it is not a stop word.  if the word is not a stop word, save it for later use.
for word in tqdm(wordlist):
    if word.casefold() not in stop_words:
         filtered_list.append(word)

print(f"\nThere are {len(filtered_list)} remaining words after cleaning them up.")


print("")
print(f"Word Frequency (top {WORD_FREQ} most used words)")
fq=FreqDist(filtered_list)
print(fq.most_common(WORD_FREQ))

#let's plot the most 10 common words
print("")

## Creating FreqDist for whole BoW, keeping the 20 most common tokens
all_fdist = FreqDist(filtered_list).most_common(WORD_FREQ)

## Conversion to Pandas series via Python Dictionary for easier plotting
all_fdist = pd.Series(dict(all_fdist))

## Setting figure, ax into variables
fig, ax = plt.subplots(figsize=(FIGURE_WIDTH,FIGURE_HEIGHT))

## Seaborn plotting using Pandas attributes + xtick rotation for ease of viewing
all_plot = sns.barplot(x=all_fdist.index, y=all_fdist.values, ax=ax)
plt.xticks(rotation=30);


# Example of Stem and Lemmatization

In [ ]:
# Initialize variables for NLTK specific functions
wnl = WordNetLemmatizer()
stemmer = PorterStemmer()

# Example inflections to reduce
example_words = ["program","programming","programer","programs","programmed"]

# Perform stemming and lemmatization to show the differences
print("{0:20}{1:20}{2:20}".format("--Word--","--Stem--","--Lemma--"))

for word in example_words:
   print ("{0:20}{1:20}{2:20}".format(word, stemmer.stem(word), wnl.lemmatize(word, pos="v")))

# Stem and Lemmatization In Depth

## Stemming

**Stemming** is a technique used to reduce an inflected word down to its word stem. For example, the words “programming,” “programmer,” and “programs” can all be reduced down to the common word stem “program.” In other words, “program” can be used as a synonym for the prior three inflection words.

Performing this text-processing technique is often useful for dealing with sparsity and/or standardizing vocabulary. Not only does it help with reducing redundancy, as most of the time the word stem and their inflected words have the same meaning, it also allows NLP models to learn links between inflected words and their word stem, which helps the model understand their usage in similar contexts.

**Stemming** algorithms function by taking a list of frequent prefixes and suffixes found in inflected words and chopping off the end or beginning of the word. This can occasionally result in word stems that are not real words; thus, we can affirm this approach certainly has its pros, but it’s not without its limitations.

### Advantages of Stemming
Improved model performance: Stemming reduces the number of unique words that need to be processed by an algorithm, which can improve its performance. Additionally, it can also make the algorithm run faster and more efficiently.
Grouping similar words: Words with a similar meaning can be grouped together, even if they have distinct forms. This can be a useful technique in tasks such as document classification, where it’s important to identify key topics or themes within a document.

Easier to analyze and understand: Since stemming typically reduces the size of the vocabulary, it’s much easier to analyze, compare, and understand texts. This is helpful in tasks such as sentiment analysis, where the goal is to determine the sentiment of a document.

### Disadvantages of Stemming

Overstemming / False positives: This is when a stemming algorithm reduces separate inflected words to the same word stem even though they are not related; for example, the Porter Stemmer algorithm stems  "universal", "university", and "universe" to the same word stem. Though they are etymologically related, their meanings in the modern day are from widely different domains. Treating them as synonyms will reduce relevance in search results.

Understemming / False negatives: This is when a stemming algorithm reduces inflected words to different word stems, but they should be the same. For example, the Porter Stemmer algorithm does not reduce the words “alumnus,” “alumnae,” and “alumni” to the same word stem, although they should be treated as synonyms.

Language challenges: As the target language's morphology, spelling, and character encoding get more complicated, stemmers become more difficult to design; For example, an Italian stemmer is more complicated than an English stemmer because there is a higher number of verb inflections. A Russian stemmer is even more complex due to more noun declensions.

## Lemmatization

**Lemmatization** is another technique used to reduce inflected words to their root word. It describes the algorithmic process of identifying an inflected word’s “lemma” (dictionary form) based on its intended meaning.

As opposed to stemming, **lemmatization** relies on accurately determining the intended part-of-speech and the meaning of a word based on its context. This means it takes into consideration where the inflected word falls within a sentence, as well as within the larger context surrounding that sentence, such as neighboring sentences or even an entire document.

“Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma”

In other words, to lemmatize a document typically means to “doing things correctly” since it involves using a vocabulary and performing morphological analysis of words to remove only the inflectional ends and return the base or dictionary form of a word, which is known as the “lemma.” For example, you can expect a lemmatization algorithm to map “runs,” “running,” and “ran” to the lemma, “run.”  

### Advantages of Lemmatization

Accuracy: Lemmatization does not merely cut words off as you see in stemming algorithms. Analysis of words is conducted based on the word’s POS to take context into consideration when producing lemmas. Also, lemmatization leads to real dictionary words being produced.

### Disadvantages of Lemmatization

Time-consuming: Compared to stemming, lemmatization is a slow and time-consuming process. This is because lemmatization involves performing morphological analysis and deriving the meaning of words from a dictionary.

# Cloud Map with Stop words applied

In [ ]:
########################################
#API Call (resultant is an image)
########################################
try:
    #invoke the word cloud API using the original list of lowercase, alpha, > 2 character words.
    wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color=IMG_BACKGROUND,
                          min_font_size=IMG_FONT_SIZE_MIN,
                          width=IMG_WIDTH,
                          height=IMG_HEIGHT,
                         ).generate(" ".join(wordlist))

except Exception as e:
    print(f"ERROR detected trying invoke the WordCloud call as follows: {str(e)}")

########################################
#Show the Results
########################################
#turn the image into a displayed graphic
plt.figure(figsize=(FIGURE_WIDTH,FIGURE_HEIGHT))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# Cloud Map without Stop Words Applied to Original Body of Text

In [ ]:
try:
    wordcloud = WordCloud(#stopwords=STOPWORDS,
                          background_color='white',                #options line "None" yield transparent background.
                          min_font_size=IMG_FONT_SIZE_MIN,
                          width=IMG_WIDTH,
                          height=IMG_HEIGHT,
                         ).generate(data)

except Exception as e:
    print(f"ERROR detected trying invoke the WordCloud() call as follows: {str(e)}")
########################################
#Show the Results
########################################
#turn the image into a displayed graphic
plt.figure(figsize=(FIGURE_WIDTH,FIGURE_HEIGHT))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
target_folder="./folderOnColab"
masks=["usfs_mask.jpg", "slf_mask.jpg", "force_mask.jpg", "kitty_mask.jpg", "pine_tree_mask.png", "paw_mask.jpg"]
if os.path.isdir(target_folder):
  for idx, filename in enumerate(masks):
    print(f"Copying {filename} to target folder: {target_folder}")
    subprocess.run(["gsutil", "-m" , "cp", "-r", f"gs://{BUCKET_NAME}/training-data/llm/masks/{filename}",  target_folder], check=True)
else:
    print("ERROR: Local folder not found/created.  Check the output to ensure your folder is created.")
    print(f"...target folder: {target_folder}")
    print("...if you can't find the problem contact the instructor.")


In [ ]:
###########################################
#- Images, Mask loading and setup
###########################################
#setup the plot mechanism to show the images.
# Adds a subplot at the 1st position
rows=1
cols=6
position=1
img_ary=[]
fig = plt.figure(figsize=(FIGURE_WIDTH, FIGURE_HEIGHT))
#fig = plt.figure()


#loop through the lists and print them side by side
for idx, filename in enumerate(masks):
    print(f"Processing...{filename}")
    fig.add_subplot(rows,cols, position)
    img_ary.append(Image.open(target_folder+os.sep+filename))
    # showing image
    plt.imshow(img_ary[idx])
    plt.axis('off')
    plt.title(os.path.basename(filename))
    # Adds a subplot at the next position
    position += 1


# With a Mask

Note that the "mask" is effectively an image of same size that's black and white.  Blacked out portions become the area you can populate so I took the USFS logo, inverted so the white becomes black and filled in the verbiage there.

Next I paste the transparent PNG on top of the original logo thus putting the words in the middle.

In [ ]:
########################################
#Mask loading and setup
########################################

#choose a different mask if you want different results, options are: usfs_mask.jpg, slf_mask.jpg, force_mask.jpg
target_filename="kitty_mask.jpg"                        #<- change here
the_logo=Image.open(target_folder+os.sep+target_filename)

try:
    #invert the colors of the mask image, so that the image is "reversed"
    inverted_image = PIL.ImageOps.invert(the_logo)
    # Convert the image to a numeric representation (a 3D array)
    the_mask = np.array(inverted_image)
except Exception as e:
    print(f"ERROR detected trying read the mask image and invert the colors call as follows: {str(e)}")


########################################
#Data Marshaling, NLP setup
########################################
data = " ".join(filtered_list)

########################################
#API Call
########################################
try:
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      mode="RGBA",
                      background_color=IMG_BACKGROUND, #white, black, blue, etc
                      min_font_size=IMG_FONT_SIZE_MIN,
                      width=IMG_WIDTH,
                      height=IMG_HEIGHT,
                      mask=the_mask,
                     ).generate(" ".join(wordlist))
    the_image= wordcloud.to_image()

except Exception as e:
    print(f"ERROR detected trying invoke the WordCloud() call as follows: {str(e)}")

#turn the image into a displayed graphic
plt.figure(figsize=(8.5,11))
plt.axis('off')
plt.imshow(the_image)
plt.show()

# Merge the Images into a single instance

You might want to try another image and text block on your own.

In [ ]:
# creating a image object (main image)
im1=the_logo
im2=the_image
alpha = im2.getchannel('A')

#transparency save
im1.paste(im2, (0,0), im2)

the_final_image=im1
plt.figure(figsize=(8.5,11))
plt.axis('off')
plt.imshow(the_final_image)
plt.show()

# Assignment

## Utilize NLP techniques to process your own data

+ Select one of the data filenames and read-in a different filename.
+ Observe the word frequency, stop word influence on size of data, stemming, and lemmatization results.
+ Observe the differences in the Word Cloud
+ Change the mask used to a different one and create a final output.

## Uber-Challenge
+ Load the Spotted Lantern Fly data file and the Eastern LongHorn Beetle data files either separately or joined together.
+ Repeat the experiment to see the influence of the new data.

## Pro-Tip:

Either create a new block of code that reads the 2nd file and combine the resulting variables `data = data_slf + " " + data_alb` or create an array, iterate through the target files and combine them.